In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler, PolynomialFeatures 
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression, Lasso, Ridge, BayesianRidge,ElasticNet

In [35]:
df_train = pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST.csv')
df_train

ValueError: The value of index_col couldn't be 'True'

In [3]:
df_variable = pd.read_csv('VariablesUsed.csv')
df_variable.groupby(['used'])['used'].count()

used
False    76
True     32
Name: used, dtype: int64

In [4]:
df_train = df_train.drop(['CLTV','MIP','MIT'],axis = 1)
df_test = df_test.drop(['CLTV','MIP','MIT'],axis = 1)
df_train

,LID,MRP,CHANNEL,SELLER,ORATE,CRATE,OUPB,OLTERM,ODATE,FPD,...,PTYPE,NUNITS,OCCSTAT,STATE,MSA,ZIP,CLDS,RMI,FORECLOSED,NMONTHS
0,8.118470e+11,3/1/2000,C,Suntrust Mortgage Inc.,8.375,8.375,58000,360,7/1/1999,9/1/1999,...,SF,1,P,OK,36420,730,0,N,False,12
1,9.288980e+11,2/1/2000,C,Amtrust Bank,7.500,7.500,204000,180,12/1/1999,2/1/2000,...,SF,1,P,WA,48300,988,0,N,False,21
2,2.229740e+11,1/1/2000,C,"Jpmorgan Chase Bank, Na",7.750,7.750,181000,360,12/1/1999,2/1/2000,...,SF,1,P,VA,40060,238,0,N,False,54
3,5.939500e+11,2/1/2000,B,"Bank Of America, N.A.",8.625,8.625,135000,360,1/1/2000,3/1/2000,...,PU,1,P,AZ,38060,852,0,N,False,16
4,6.773390e+11,2/1/2000,C,Other,7.750,7.750,189000,360,12/1/1999,2/1/2000,...,SF,1,P,IN,29140,479,0,N,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164570,8.194500e+11,2/1/2000,C,Amtrust Bank,7.875,7.875,106000,180,11/1/1999,1/1/2000,...,SF,1,I,MI,11460,481,0,N,False,24
164571,6.558220e+11,3/1/2000,R,"Jpmorgan Chase Bank, Na",8.000,8.000,144000,180,2/1/2000,4/1/2000,...,SF,1,P,CA,41740,920,0,N,False,19
164572,6.577330e+11,3/1/2000,R,First Tennessee Bank National Association,8.250,8.250,88000,360,2/1/2000,3/1/2000,...,SF,1,P,WV,26580,255,0,N,False,12
164573,3.145290e+11,1/1/2000,R,"Ge Mortgage Services, Llc",7.875,7.875,99000,240,10/1/1999,12/1/1999,...,SF,1,P,NY,35620,115,0,N,False,217


In [5]:
df_train.isnull().sum()

LID               0
MRP               0
CHANNEL           0
SELLER            0
ORATE             0
CRATE             0
OUPB              0
OLTERM            0
ODATE             0
FPD               0
LA                0
TMTLM             0
RMTM              0
MDATE             0
LOTV              1
NB              198
DTI            6998
BCSAO          3981
CBCSAO        82221
FTHBI           218
LP                0
PTYPE             0
NUNITS            0
OCCSTAT           0
STATE             0
MSA               0
ZIP               0
CLDS              0
RMI               0
FORECLOSED        0
NMONTHS           0
dtype: int64

In [6]:
X_train = df_train.iloc[:,1:-2]
X_test = df_test.iloc[:,1:]
Y_train = df_train['NMONTHS']
Y_train = Y_train.to_numpy()
names_train = X_train.columns.values
names_test = X_test.columns.values

In [7]:
categorical_cols = ['CHANNEL','SELLER','FTHBI','LP','PTYPE','OCCSTAT','STATE','RMI']

In [8]:
for col in X_train.columns:
    if col in categorical_cols:
        a={}
        for i in range(len(X_train[col].unique())):
            a[X_train[col].unique()[i]]=i
        work=X_train[col].map(a)
        X_train[col]=work
#encoded = list(X_train.columns)

for col in X_test.columns:
    if col in categorical_cols:
        a={}
        for i in range(len(X_test[col].unique())):
            a[X_test[col].unique()[i]]=i
        work=X_test[col].map(a)
        X_test[col]=work
#encoded = list(X_test.columns)

In [9]:
Date = ['MRP','ODATE','FPD']
for i in Date:
    X_train[i]= pd.to_datetime(X_train[i])
    X_test[i]= pd.to_datetime(X_test[i])
    X_train[i] =(X_train[i]-min(X_train[i])).dt.days
    X_test[i] =(X_test[i]-min(X_test[i])).dt.days

In [10]:
X_missing_train = X_train[['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI']]
X_missing_test = X_test[['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI']]

In [11]:
knn_imputer = KNNImputer(n_neighbors = 2)
X_filled_train = knn_imputer.fit_transform(X_missing_train)
X_filled_train = pd.DataFrame(X_filled_train,columns = ['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI'])
X_filled_test = knn_imputer.fit_transform(X_missing_test)
X_filled_test = pd.DataFrame(X_filled_test,columns = ['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI'])

In [12]:
train_A = X_train.iloc[:,0:13]
train_B = X_train.iloc[:,19:]
X_train = pd.concat([train_A,X_filled_train,train_B], axis=1)
test_A = X_test.iloc[:,0:13]
test_B = X_test.iloc[:,19:]
X_test = pd.concat([test_A,X_filled_test,test_B], axis=1)

In [13]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [14]:
len(np.transpose(X_test))

28

Regression

In [15]:
models = {"Logistic": LogisticRegression(n_jobs=-1), 
            "Liear": LinearRegression(n_jobs=-1),
            "Lasso": Lasso(),
            "Ridge": Ridge(),
             "BayesianRidge": BayesianRidge(),
             "ElasticNet" : ElasticNet()}  

In [16]:
for model_name, model in models.items():
    model.fit(X_train, Y_train)
    pred = model.predict(X_test)
    r_sq = model.score(X_train, Y_train)
    print("Model: {}, r_sq: {:.2f}".format(model_name, r_sq))

Model: Logistic, r_sq: 0.05
Model: Liear, r_sq: 0.10
Model: Lasso, r_sq: 0.09
Model: Ridge, r_sq: 0.10
Model: BayesianRidge, r_sq: 0.10
Model: ElasticNet, r_sq: 0.08


In [17]:
pca = PCA(n_components=18)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(sum(pca.explained_variance_ratio_))

0.8935819988652844


In [18]:
for model_name, model in models.items():
    model.fit(X_train_pca, Y_train)
    pred_pca = model.predict(X_test_pca)
    r_sq = model.score(X_train_pca, Y_train)
    print("Model: {}, r_sq: {:.2f}".format(model_name, r_sq))

Model: Logistic, r_sq: 0.05
Model: Liear, r_sq: 0.07
Model: Lasso, r_sq: 0.06
Model: Ridge, r_sq: 0.07
Model: BayesianRidge, r_sq: 0.07
Model: ElasticNet, r_sq: 0.06


In [19]:
pred

array([28.80249713, 34.81039625, 38.45919092, ..., 18.17584884,
       40.11261394, 33.77865983])

In [39]:
import patsy as ps
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import patsy as ps

data["LNMONTH"]=data.NMONTHS.apply(np.log)
print(data.head())
N=data.shape[0]
perm=np.random.permutation(range(N))
data=data.loc[perm]
Ntrain=int(.8*N)
Ntest=N-Ntrain
data_train=data.iloc[0:Ntrain]
data_test=data.iloc[Ntrain:N]

            LID       MRP CHANNEL                   SELLER  ORATE  CRATE  \
0  8.118470e+11  3/1/2000       C   Suntrust Mortgage Inc.  8.375  8.375   
1  9.288980e+11  2/1/2000       C             Amtrust Bank  7.500  7.500   
2  2.229740e+11  1/1/2000       C  Jpmorgan Chase Bank, Na  7.750  7.750   
3  5.939500e+11  2/1/2000       B    Bank Of America, N.A.  8.625  8.625   
4  6.773390e+11  2/1/2000       C                    Other  7.750  7.750   

     OUPB  OLTERM      ODATE       FPD  ...  STATE    MSA  ZIP   MIP  CLDS  \
0   58000     360   7/1/1999  9/1/1999  ...     OK  36420  730   NaN     0   
1  204000     180  12/1/1999  2/1/2000  ...     WA  48300  988   NaN     0   
2  181000     360  12/1/1999  2/1/2000  ...     VA  40060  238  30.0     0   
3  135000     360   1/1/2000  3/1/2000  ...     AZ  38060  852  30.0     0   
4  189000     360  12/1/1999  2/1/2000  ...     IN  29140  479   NaN     0   

   MIT  RMI  FORECLOSED  NMONTHS   LNMONTH  
0  NaN    N       False      

In [40]:
formula="LNMONTH~MRP+CHANNEL+SELLER+ORATE+CRATE+OUPB+OLTERM+ODATE+FPD+LA+TMTLM+RMTM+MDATE+LOTV+NB+DTI+BCSAO+CBCSAO+FTHBI+LP+PTYPE+NUNITS+OCCSTAT+STATE+MSA+ZIP+CLDS+RMI"
Ytrain,Xtrain=ps.dmatrices(formula,data_train)
Ytest,Xtest=ps.dmatrices(formula,data_test)

In [41]:
Ntest=Ytest.shape[0]
reg= LinearRegression().fit(Xtrain,Ytrain)
Ytest_pred=reg.predict(Xtest)
e=pd.Series((Ytest-Ytest_pred).reshape((Ntest,)))
MAD=e.apply(abs).mean()
print(MAD)
print(Xtrain)

ValueError: X has 135 features, but LinearRegression is expecting 134 features as input.

NameError: name 'X' is not defined